# NetPyNE Tutorial 9: Evolutionary Algorithm for Parameters Optimization

## Preliminaries

If you are going to run this notebook locally using Jupyter Notebook, start from following instructions https://github.com/suny-downstate-medical-center/netpyne/blob/development/netpyne/tutorials/README.md.

If you are using Open Source Brain or EBRAINS, everything is already set up.

On any other online platform (e.g. on Google Collab) you might need to run the following commmands to install NEURON and NetPyNE using **pip**:
```
!pip install neuron
!pip install netpyne
```



Code in this tutorial provides the set up of batch simulation itself, while the network paramters and default configuration of individual simulations are provided in separate files in `tut_batch_evol` folder. We will switch to this folder to make these files accessible from this notebook. We also need to install `inspyred`, which implements the core of evolutionary algorithm.

In [1]:
!pip install inspyred
import os
os.chdir(os.getcwd()+'/tut_batch_evol')

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)




Two examples are provided: 'simple' and 'complex'.
In 'simple', 3 parameters are optimized to match target firing rates in 2 populations.
In 'complex', 6 parameters are optimized to match target firing rates in 6 populations.

First we import all necessary modules, and then describe parameters space to explore.

In [2]:
# %matplotlib inline
from netpyne import specs
from netpyne.batch import Batch

networkType = "simple"  # 'simple' or 'complex'

if networkType == "simple":
    ## simple net
    params = specs.ODict()
    params["prob"] = [0.01, 0.5]
    params["weight"] = [0.001, 0.1]
    params["delay"] = [1, 20]

    pops = {}
    pops["S"] = {"target": 5, "width": 2, "min": 2}
    pops["M"] = {"target": 15, "width": 2, "min": 0.2}

elif networkType == "complex":
    # complex net
    params = specs.ODict()
    params["probEall"] = [0.05, 0.2]  # 0.1
    params["weightEall"] = [0.0025, 0.0075]  # 5.0
    params["probIE"] = [0.2, 0.6]  # 0.4
    params["weightIE"] = [0.0005, 0.002]
    params["probLengthConst"] = [100, 200]
    params["stimWeight"] = [0.05, 0.2]

    pops = {}
    pops["E2"] = {"target": 5, "width": 2, "min": 1}
    pops["I2"] = {"target": 10, "width": 5, "min": 2}
    pops["E4"] = {"target": 30, "width": 10, "min": 1}
    pops["I4"] = {"target": 10, "width": 3, "min": 2}
    pops["E5"] = {"target": 40, "width": 4, "min": 1}
    pops["I5"] = {"target": 25, "width": 5, "min": 2}

--No graphics will be displayed.


Provide fitness function and it's arguments:

In [3]:
fitnessFuncArgs = {}
fitnessFuncArgs["pops"] = pops
fitnessFuncArgs["maxFitness"] = 1000


def fitnessFunc(simData, **kwargs):
    import numpy as np

    pops = kwargs["pops"]
    maxFitness = kwargs["maxFitness"]
    popFitness = [None for i in pops.items()]
    popFitness = [
        (
            min(
                np.exp(abs(v["target"] - simData["popRates"][k]) / v["width"]),
                maxFitness,
            )
            if simData["popRates"][k] > v["min"]
            else maxFitness
        )
        for k, v in pops.items()
    ]
    fitness = np.mean(popFitness)
    popInfo = "; ".join(
        [
            "%s rate=%.1f fit=%1.f" % (p, r, f)
            for p, r, f in zip(
                list(simData["popRates"].keys()),
                list(simData["popRates"].values()),
                popFitness,
            )
        ]
    )
    print("  " + popInfo)
    return fitness

Here we create `Batch` object with paramaters to tune, and set output folder, optimization method ('evol'), fitness function and all the configurations from above.

In [4]:
batch = Batch(params=params)

# Set output folder, grid method (all param combinations), and run configuration
batch.batchLabel = "simple"
batch.saveFolder = "./" + batch.batchLabel
batch.method = "evol"
batch.runCfg = {
    "type": "mpi_bulletin",  #'hpc_slurm',
    "script": "init.py",
    # options required only for hpc
    "mpiCommand": "mpirun",
    "nodes": 1,
    "coresPerNode": 2,
    "allocation": "default",
    "email": "salvadordura@gmail.com",
    "reservation": None,
    "folder": "/home/salvadord/evol",
    #'custom': 'export LD_LIBRARY_PATH="$HOME/.openmpi/lib"' # only for conda users
}
batch.evolCfg = {
    "evolAlgorithm": "custom",
    "fitnessFunc": fitnessFunc,  # fitness expression (should read simData)
    "fitnessFuncArgs": fitnessFuncArgs,
    "pop_size": 6,
    "num_elites": 1,  # keep this number of parents for next generation if they are fitter than children
    "mutation_rate": 0.4,
    "crossover": 0.5,
    "maximize": False,  # maximize fitness function?
    "max_generations": 4,
    "time_sleep": 5,  # wait this time before checking again if sim is completed (for each generation)
    "maxiter_wait": 40,  # max number of times to check if sim is completed (for each generation)
    "defaultFitness": 1000,  # set fitness value in case simulation time is over
}

Now we can run this code. When completed, you can inspect folder `tut_batch_evol/{networkType}`, where `{networkType}` is either `simple` or `complex`, to find the output of each generation and the summary for all, including the optimal values of requested parameters (short summary will also appear in the output of the cell). 

Note that the algorithm is stochastic, so you will not get the same results if re-run it again.

In [5]:
batch.run()

Saving batch to ./simple/simple_batch.json ... 
['prob', 'weight', 'delay'] [0.2184408472918326, 0.0762105862358311, 11.596004405602478]
set prob=0.2184408472918326
set weight=0.0762105862358311
set delay=11.596004405602478
Saving simConfig to ./simple/gen_0/gen_0_cand_0_cfg.json ... 
--------------------------------------------------------------------------------
['prob', 'weight', 'delay'] [0.4548478651105395, 0.031485895166197285, 11.489959979459424]
set prob=0.4548478651105395
set weight=0.031485895166197285
set delay=11.489959979459424
Saving simConfig to ./simple/gen_0/gen_0_cand_1_cfg.json ... 
--------------------------------------------------------------------------------
['prob', 'weight', 'delay'] [0.44562492993940084, 0.0025818549371395653, 17.819305299364263]
set prob=0.44562492993940084
set weight=0.0025818549371395653
set delay=17.819305299364263
Saving simConfig to ./simple/gen_0/gen_0_cand_2_cfg.json ... 
----------------------------------------------------------------

SystemExit: 